In [3]:
import yfinance as yf
from datetime import datetime, timedelta

symbol = 'AAPL'
end_date = datetime.now()

def get_stock_data(date, interval):
    stock_data = yf.download(symbol, start=date, end=end_date, interval=interval)
    stock_data = get_sma_data(stock_data)
    stock_data = get_stock_volume(stock_data)
    
    return stock_data

def get_sma_data(stock_data):
    stock_data['Sma'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['Std'] = stock_data['Close'].rolling(window=20).std()
    stock_data['Upper'] = stock_data['Sma'] + (stock_data['Std'] * 2)
    stock_data['Lower'] = stock_data['Sma'] - (stock_data['Std'] * 2)
    
    return stock_data

def get_stock_volume(stock_data):
    stock_data['Volume'] = stock_data['Volume'].fillna(0)
    
    return stock_data

def stock_data_to_json(stock_data, interval):
    stock_data = stock_data.drop(columns=['Adj Close', 'Std'])
    json_data = stock_data.reset_index().to_dict(orient='records')
    index_name = 'Date' if 'Date' in stock_data.index.names else 'Datetime'
    
    for record in json_data:
        if interval == '1m' or interval == '1h':
            record[index_name] = record[index_name].strftime('%Y-%m-%d %H:%M')
        else:
            record[index_name] = record[index_name].strftime('%Y-%m-%d')
        
    return json_data

stock_data_list = []

start_date = end_date - timedelta(days=1)
stockdata = get_stock_data(start_date, '1m')
stock_data_list.append(stock_data_to_json(stockdata, '1m'))

start_date = end_date - timedelta(days=30)
stockdata = get_stock_data(start_date, '1h')
stock_data_list.append(stock_data_to_json(stockdata, '1h'))

start_date = end_date - timedelta(days=180)
stockdata = get_stock_data(start_date, '1d')
stock_data_list.append(stock_data_to_json(stockdata, '1d'))

start_date = end_date - timedelta(days=365)
stockdata = get_stock_data(start_date, '1d')
stock_data_list.append(stock_data_to_json(stockdata, '1d'))

start_date = end_date - timedelta(days=365 * 5)
stockdata = get_stock_data(start_date, '5d')
stock_data_list.append(stock_data_to_json(stockdata, '5d'))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
stock_data = yf.download(
            'BTC-USD', start=datetime.now()-timedelta(days=1), end=end_date, interval='1m')
stock_data

'''
I just tested a script using ydownload with BTC-USD and an interval of 1m and I get the same result. If I had to guess, Yahoo isn't offering OHLC data at this level at the moment.
'''